# Thermal rates calculation and comparison

This notebook calculates and compares the thermal rates using data from C. Vogl's version of TARDIS.

In [1]:
from pathlib import Path

import pandas as pd

electron_temp = 9992.2722969523056
electron_density = 2.20676447e09
ff_heating_estimator = [  4.89135279e-24,   4.37696370e-24,   3.75869301e-24,
         4.97847160e-24,   4.52158002e-24,   4.21024499e-24,
         3.94991540e-24,   3.72915649e-24,   3.58902110e-24,
         3.40170224e-24,   3.20848519e-24,   3.03540032e-24,
         2.87314722e-24,   2.74328938e-24,   2.61063140e-24,
         2.50640248e-24,   2.38164559e-24,   2.26967531e-24,
         2.24509826e-24,   2.12378192e-24,   2.02063266e-24,
         1.92509873e-24,   1.83070678e-24,   1.77346374e-24]

home = Path.home()

# example data from C Vogl
data_path = home / "tardis-regression-data/testdata/thermal_data"
bf_heating_estimator = pd.read_csv(data_path / "thermal_bf_heating_est.csv", index_col=(0, 1, 2))
stim_recomb_cooling_estimator = pd.read_csv(data_path / "thermal_stim_cooling_est.csv", index_col=(0, 1, 2))
ion_number_density = pd.read_csv(data_path / "thermal_ion_number_density.csv", index_col=(0, 1))
level_number_density = pd.read_csv(data_path / "thermal_level_number_density.csv", index_col=(0, 1, 2))
level_population_ratio = pd.read_csv(data_path / "thermal_level_pop_ratio.csv", index_col=(0, 1, 2))
coll_exc_coeff = pd.read_csv(data_path / "thermal_coll_exc_coeff.csv", index_col=(0, 1, 2, 3))
coll_deexc_coeff = pd.read_csv(data_path / "thermal_coll_deexc_coeff.csv", index_col=(0, 1, 2, 3))
lines = pd.read_csv(data_path / "thermal_lines.csv", index_col=(0))
coll_ion_rate_coeff = pd.read_csv(data_path / "thermal_coll_ion_rate_coeff.csv", index_col=(0, 1, 2))

# because pandas reads in the columns as strings, we need to convert them back to integers
bf_heating_estimator.columns = bf_heating_estimator.columns.astype(int)
stim_recomb_cooling_estimator.columns = stim_recomb_cooling_estimator.columns.astype(int)
ion_number_density.columns = ion_number_density.columns.astype(int)
level_number_density.columns = level_number_density.columns.astype(int)
level_population_ratio.columns = level_population_ratio.columns.astype(int)
coll_exc_coeff.columns = coll_exc_coeff.columns.astype(int)
coll_deexc_coeff.columns = coll_deexc_coeff.columns.astype(int)
lines.index = lines.index.astype(int)
coll_ion_rate_coeff.columns = coll_ion_rate_coeff.columns.astype(int)

In [2]:
#Benchmark values from ctardis

fb_cooling_ct = 1.2018593543520837e-06
ff_cooling_ct = 6.941664530316456e-07
coll_ion_cooling_ct = 1.6125333965984259e-07
bf_heating_ct = 1.2809489753862688e-06
ff_heating_ct = 2.3829164962085199e-07
coll_ion_heating_ct = 1.5946196993219911e-07
total_heating_rate_ct = -3.8611749800759465e-07
fractional_heating_rate_ct = -0.036553097452112299

col_exc_cooling_rate_ct = 8.5059159013e-06
col_deexc_heating_rate_ct = 8.49837495539e-06

In [3]:
import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.plasma.electron_energy_distribution import (
         ThermalElectronEnergyDistribution,
)
from tardis.plasma.equilibrium.rates.heating_cooling_rates import (
         BoundFreeThermalRates,
         CollisionalBoundThermalRates,
         CollisionalIonizationThermalRates,
         FreeFreeThermalRates,
)

atom_data = AtomData.from_hdf(home / "tardis-regression-data/atom_data/nlte_atom_data/TestNLTE_He_Ti_ctardis.h5") # currently not available for public use
atom_data.prepare_atom_data([1], "macroatom", [], [(1, 0)])


thermal_electron_distribution = ThermalElectronEnergyDistribution(0 * u.erg, electron_temp * u.K, electron_density * u.cm**-3)

/home/afullard/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [4]:
bf_rates = BoundFreeThermalRates(atom_data.photoionization_data)
bf_heating, bf_cooling = bf_rates.solve(
    level_number_density,
    ion_number_density,
    thermal_electron_distribution,
    level_population_ratio,
    bound_free_heating_estimator=bf_heating_estimator,
    stimulated_recombination_estimator=stim_recomb_cooling_estimator,
)

In [5]:
(bf_heating.iloc[0] - bf_heating_ct) / bf_heating_ct

np.float64(2.4565595191649485e-13)

In [6]:
(bf_cooling.iloc[0] - fb_cooling_ct) / fb_cooling_ct

np.float64(0.8467889208007834)

In [7]:
ff_rates = FreeFreeThermalRates()
ff_heating, ff_cooling = ff_rates.solve(
    ff_heating_estimator[0],
    thermal_electron_distribution,
    ion_number_density,
)

In [8]:
(ff_cooling.iloc[0] - ff_cooling_ct) / ff_cooling_ct

np.float64(-2.8964722536676576e-11)

In [9]:
(ff_heating.iloc[0] - ff_heating_ct) / ff_heating_ct

np.float64(-9.884951847750145e-10)

In [10]:
coll_ion_rates = CollisionalIonizationThermalRates(atom_data.photoionization_data)
coll_ion_heating, coll_ion_cooling = coll_ion_rates.solve(
    thermal_electron_distribution.number_density,
    ion_number_density,
    level_number_density,
    coll_ion_rate_coeff,
    level_population_ratio,
    )

In [11]:
(coll_ion_heating.iloc[0] - coll_ion_heating_ct) / coll_ion_heating_ct

np.float64(0.0004125934943592712)

In [12]:
(coll_ion_cooling.iloc[0] - coll_ion_cooling_ct) / coll_ion_cooling_ct

np.float64(0.00041378496294871474)

In [13]:
# need to get data with collision rates
coll_bound_rates = CollisionalBoundThermalRates(lines)
coll_deexc_heating, coll_exc_cooling = coll_bound_rates.solve(
    thermal_electron_distribution.number_density,
    coll_deexc_coeff,
    coll_exc_coeff,
    level_number_density,
)

In [14]:
(coll_deexc_heating.iloc[0] - col_deexc_heating_rate_ct) / col_deexc_heating_rate_ct

np.float64(-0.00024325566644572874)

In [15]:
(coll_exc_cooling.iloc[0] - col_exc_cooling_rate_ct) / col_exc_cooling_rate_ct

np.float64(-0.0002385020503019561)